In [1]:
import numpy as np
import pickle
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
import os
import sys


In [2]:
def handle_helper_functions():
    print(sys.path)
    directory_path = os.path.abspath(os.path.join('../../helper'))
    if directory_path not in sys.path:
        sys.path.append(directory_path)    

    translation_path=os.path.abspath(os.path.join('../../utils')) 
    if translation_path not in sys.path:
        sys.path.append(translation_path)   
    sys.path.append('/Users/learn/Desktop/Projects/machine-translation/test/helper')                                       
    sys.path.append('/Users/learn/Desktop/Projects/machine-translation/utils')
    sys.path.append('/Users/learn/Desktop/Projects/machine-translation/data')


    lstm_helper_path = os.path.abspath(os.path.join('./create-model/lstm/'))
    if lstm_helper_path not in sys.path:
        sys.path.append(lstm_helper_path)    

    print(sys.path)
    

handle_helper_functions()

['/Users/learn/Desktop/Projects/machine-translation/tutorial', '/Users/learn/.vscode/extensions/ms-toolsai.jupyter-2022.11.1003412109/pythonFiles', '/Users/learn/.vscode/extensions/ms-toolsai.jupyter-2022.11.1003412109/pythonFiles/lib/python', '/Users/learn/Desktop/Projects/machine-translation/.env/lib/python39.zip', '/Users/learn/Desktop/Projects/machine-translation/.env/lib/python3.9', '/Users/learn/Desktop/Projects/machine-translation/.env/lib/python3.9/lib-dynload', '', '/Users/learn/Desktop/Projects/machine-translation/.env/lib/python3.9/site-packages']
['/Users/learn/Desktop/Projects/machine-translation/tutorial', '/Users/learn/.vscode/extensions/ms-toolsai.jupyter-2022.11.1003412109/pythonFiles', '/Users/learn/.vscode/extensions/ms-toolsai.jupyter-2022.11.1003412109/pythonFiles/lib/python', '/Users/learn/Desktop/Projects/machine-translation/.env/lib/python39.zip', '/Users/learn/Desktop/Projects/machine-translation/.env/lib/python3.9', '/Users/learn/Desktop/Projects/machine-trans

In [3]:
def load_models_and_parameters(model_size):

    path=model_size+'/'

    from tensorflow import keras
    model = keras.models.load_model(path+'lstm_model')
    with open(path+ "src_parameters.pickle", 'rb') as handle:
        src_parameters = pickle.load(handle)

    with open(path+ "src_tokenizer.pickle", 'rb') as handle:
        src_tokenizer = pickle.load(handle)

    with open(path+ "target_parameters.pickle", 'rb') as handle:
        target_parameters = pickle.load(handle)

    with open(path+ "target_tokenizer.pickle", 'rb') as handle:
        target_tokenizer = pickle.load(handle)
    return model, src_tokenizer, target_tokenizer, src_parameters, target_parameters


model, src_tokenizer, target_tokenizer, src_parameters, target_parameters= load_models_and_parameters('10000')

2023-01-15 20:22:32.467709: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-01-15 20:22:32.769624: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-01-15 20:22:32.776718: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-01-15 20:22:32.831300: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-01-15 20:22:33.281298: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_sh

In [4]:

src_length=src_parameters["src_length"]
src_vocab_size=src_parameters["src_vocab_size"]

target_length=target_parameters["target_length"]
target_vocab_size=target_parameters["target_vocab_size"]

print(src_length, target_length, src_vocab_size, target_vocab_size)




15 15 2384 1911


In [5]:
from iit_dataset import createDataset

Using custom data configuration cfilt--iitb-english-hindi-911387c6837f8b91
Reusing dataset parquet (/Users/learn/.cache/huggingface/datasets/parquet/cfilt--iitb-english-hindi-911387c6837f8b91/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
data_size=1000

pool_oftexts, pairs =createDataset(data_size=data_size, type="test")
dataset= pool_oftexts
dataset = dataset.values
test=dataset

In [7]:
source_str, target_str = "Hindi", "English"


In [8]:
def encode_sequences(tokenizer, length, lines):
    # encode and pad sequences
    X = tokenizer.texts_to_sequences(lines) # integer encode sequences
    X = pad_sequences(X, maxlen=length, padding='post') # pad sequences with 0 values
    return X
 
def encode_output(sequences, vocab_size):
    # one hot encode target sequence
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = np.array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y

In [9]:
def word_for_id(integer, tokenizer):
    # map an integer to a word
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None
 
def predict_seq(model, tokenizer, source):
    # generate target from a source sequence
    prediction = model.predict(source, verbose=0)[0]
    integers = [np.argmax(vector) for vector in prediction]
    target = list()
    for i in integers:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)

def compare_prediction(model, tar_tokenizer, sources, raw_dataset, limit=20):
    # evaluate a model
    actual, predicted = [], []
    src = f'{source_str.upper()} (SOURCE)'
    tgt = f'{target_str.upper()} (TARGET)'
    pred = f'AUTOMATIC TRANSLATION IN {target_str.upper()}'
    print(f'{src:30} {tgt:25} {pred}\n')
    
    for i, source in enumerate(sources): # translate encoded source text
        source = source.reshape((1, source.shape[0]))
        translation = predict_seq(model, tar_tokenizer, source)
        raw_src,raw_target = raw_dataset[i]
        print(f' {i+1}. {raw_src:30} || {raw_target:25} || {translation}')
        if i >= limit: # Display some of the result
            break
 

# test on some test sequences
print('\n\n### Result on the Test Set ###')
# compare_prediction(model, tar_tokenizer, testX, test)



### Result on the Test Set ###


In [10]:
# Prepare test data
idx_src=0
idx_tar=1
testX = encode_sequences(src_tokenizer, src_length, test[:, idx_src])
testY = encode_sequences(target_tokenizer, target_length, test[:, idx_tar])
testY = encode_output(testY, target_vocab_size)

In [11]:
print('\n\n### Result on the Test Set ###')
compare_prediction(model, target_tokenizer, testX, test)



### Result on the Test Set ###
HINDI (SOURCE)                 ENGLISH (TARGET)          AUTOMATIC TRANSLATION IN ENGLISH



2023-01-15 20:22:40.488924: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


 1. आपकी कार में ब्लैक बॉक्स       || a black box in your car   || make stack
 2. जबकि अमेरिका के सड़क योजनाकार ध्वस्त होते हुए हाईवे सिस्टम को सुधारने के लिए धन की कमी से जूझ रहे हैं वहीं बहुतसे लोग इसका समाधान छोटे से ब्लैक बॉक्स में देख रहे हैं जो आपकी कार के डैशबोर्ड पर सफ़ाई से फिट हो जाता है। || as americas road planners struggle to find the cash to mend a crumbling highway system many are beginning to see a solution in a little black box that fits neatly by the dashboard of your car || the are for the
 3. यह डिवाइस जो मोटरचालक द्वारा वाहन चलाए गए प्रत्येक मील को ट्रैक करती है तथा उस सूचना को अधिकारियों को संचारित करती है आजकल अमेरिका की प्रमुख सड़कों का वित्तपोषण करने के लिए पुराने हो चुके सिस्टम का जीर्णोद्धार करने के लिए वाशिंगटन और राज्य नियोजन कार्यालय के लिए एक विवादास्पद प्रयास का मुद्दा बन चुका है। || the devices which track every mile a motorist drives and transmit that information to bureaucrats are at the center of a controversial attempt in washington and state planni